### <b> VERSION 3 </b>


##### La idea es ahora a partir de los temas vistos en clase poder hacer feature engineering de una manera mas objetiva

In [34]:
import numpy as np
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

### Asi voy a dividir entre train y test para mi validacion

In [22]:
def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.35)
    return x_train, x_test, y_train, y_test

# x_train y_train son mi TRAIN
# x_test y_test son mi VALIDACION

### ................................................................................................................................

### <b> Modelo N°1 - XGBoost </b>

In [23]:
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

#### Para sacar provecho a las labels provincia, tipodepropiedad y ciudad usaré algún tipo de Mean Target Encoding (basado en el promedio del precio para cada grupo de valores)

In [24]:
def mean_target_encoding(train, nombrefeat, nombretarget):
        
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean) #Saco promedio
    temp = preprocessing.scale(temp) #Normalizo
    train[nombrefeat + "_mean"] = temp #Dejo en el DF
    
    return train

In [25]:
def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

In [26]:
def armar_set_2(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo', 'direccion', 'fecha', 'id', 'antiguedad',
                        'lng', 'tipodepropiedad'], axis = 1)
    
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    #Elimino los residuos
    train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                        'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train

    temporal = x_train
    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "provincia", "precio")
    x_train_mean['precio'] = y_train
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "provincia", temporal)
    x_test = mean_target_decoding(x_test, "ciudad", temporal)
    
    x_train_mean.drop(["precio","provincia", "ciudad"], axis=1, inplace = True)
    x_test.drop(["provincia", "ciudad"], axis=1, inplace = True)

    
    return x_train_mean, x_test, y_train, y_test

In [33]:
x_train, x_test, y_train, y_test = armar_set_2(train)

#### Sobre este modelo hago un grid search

In [28]:
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [38]:
# A parameter grid for XGBoost
params = {'gamma':[i/60.0 for i in range(4,6)],  
          'subsample':[i/15.0 for i in range(9,11)],
          'colsample_bytree': [0.7], 
          'max_depth': [9,10],
          'n_estimators' : [500, 600, 700],
          'learning_rate' : [i/80.0 for i in range(3,6)]
         }


# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params, cv=3)
grid.fit(x_train, y_train)

predicciones = grid.best_estimator_.predict(x_test)

# Print the MAE
print(mean_absolute_error(y_test, predicciones))

[15:44:49] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/lorenzo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:45:45] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/lorenzo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:46:52] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[19:48:27] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/lorenzo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[19:49:47] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/lorenzo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[19:51:08] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
608803.2456041666


In [39]:
grid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7,
             gamma=0.06666666666666667, importance_type='gain',
             learning_rate=0.0375, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=None, n_estimators=700, n_jobs=1,
             nthread=-1, objective='reg:linear', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=0.6666666666666666, verbosity=1)

In [41]:
#Anterior: 617944.20


#XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
 #            colsample_bynode=1, colsample_bytree=0.7, gamma=0.08,
 #            importance_type='gain', learning_rate=0.06666666666666667,
 #            max_delta_step=0, max_depth=9, min_child_weight=1, missing=None,
 #            n_estimators=500, n_jobs=1, nthread=-1, objective='reg:linear',
 #            random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
 #            seed=None, silent=None, subsample=0.6, verbosity=1)
#

#### GRID SEARCH ####

#params = {'gamma':[i/60.0 for i in range(4,6)],    --------------> Tomó el 4
#          'subsample':[i/15.0 for i in range(9,11)], ------------> Tomó el 10
#          'colsample_bytree': [0.7], 
#          'max_depth': [9,10],                      -------------> Tomó el 10
#          'n_estimators' : [500, 600, 700],         -------------> Tomó el 700
#          'learning_rate' : [i/80.0 for i in range(3,6)] --------> Tomó el 3
#         }

#Nuevo: 608803.24

#XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#             colsample_bynode=1, colsample_bytree=0.7,
#             gamma=0.06666666666666667, importance_type='gain',
#             learning_rate=0.0375, max_delta_step=0, max_depth=10,
#             min_child_weight=1, missing=None, n_estimators=700, n_jobs=1,
#             nthread=-1, objective='reg:linear', random_state=0, reg_alpha=0,
#             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
#             subsample=0.6666666666666666, verbosity=1)



# A parameter grid for XGBoost
params = {'gamma':[i/60.0 for i in range(2,5)],  
          'subsample':[10/15.0],
          'colsample_bytree': [0.7], 
          'max_depth': [10,11],
          'n_estimators' : [700,800,900],
          'learning_rate' : [i/80.0 for i in range(2,4)]
         }


# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params, cv=3)
grid.fit(x_train, y_train)

predicciones = grid.best_estimator_.predict(x_test)

# Print the MAE
print(mean_absolute_error(y_test, predicciones))

[20:58:15] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/lorenzo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[20:59:35] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/lorenzo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:01:03] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[00:37:22] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/lorenzo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[00:39:26] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/lorenzo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[00:41:25] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
602961.5166218377


In [42]:
grid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7,
             gamma=0.03333333333333333, importance_type='gain',
             learning_rate=0.025, max_delta_step=0, max_depth=11,
             min_child_weight=1, missing=None, n_estimators=900, n_jobs=1,
             nthread=-1, objective='reg:linear', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=0.6666666666666666, verbosity=1)

In [ ]:
#ULTIMO:

# 602961.5166218377

# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#             colsample_bynode=1, colsample_bytree=0.7,
#             gamma=0.03333333333333333, importance_type='gain',
#             learning_rate=0.025, max_delta_step=0, max_depth=11,
#             min_child_weight=1, missing=None, n_estimators=900, n_jobs=1,
#             nthread=-1, objective='reg:linear', random_state=0, reg_alpha=0,
#             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
#             subsample=0.6666666666666666, verbosity=1)#

In [ ]:
#Armo modelo
import xgboost as xgb

xgb_model = xgb.XGBRegressor(base_score=0.5, 
                             booster='gbtree', 
                             colsample_bylevel=1,
                             colsample_bynode=1, 
                             colsample_bytree=0.7, 
                             gamma=0.2,
                             importance_type='gain', 
                             learning_rate=0.1, 
                             max_delta_step=0,
                             max_depth=9, 
                             min_child_weight=4, 
                             missing=None, 
                             n_estimators=300,
                             n_jobs=1, 
                             nthread=-1, 
                             objective='reg:linear', 
                             random_state=0,
                             reg_alpha=0, 
                             reg_lambda=1, 
                             scale_pos_weight=1, 
                             seed=None,
                             silent=None, 
                             subsample=0.7, 
                             verbosity=1)

#Fitteo
xgb_model.fit(x_train, y_train)

#Predigo
prediccion = xgb_model.predict(x_test)

mae = mean_absolute_error(y_test, prediccion)

print("Promedio de error absoluto obtenido: " + str(mae))